In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/diabetes.csv')

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=43)

In [ ]:
(X_train.shape,y_train.shape)

((614, 8), (614,))

In [ ]:
(X_test.shape,y_test.shape)

((154, 8), (154,))

In [ ]:
import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense,Input

In [ ]:
model = Sequential()
model.add(Input(shape=(8,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6694 - loss: 0.6434 - val_accuracy: 0.7143 - val_loss: 0.5985
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7109 - loss: 0.5963 - val_accuracy: 0.7143 - val_loss: 0.5701
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7351 - loss: 0.5710 - val_accuracy: 0.7403 - val_loss: 0.5504
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7435 - loss: 0.5376 - val_accuracy: 0.7273 - val_loss: 0.5361
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7778 - loss: 0.5184 - val_accuracy: 0.7338 - val_loss: 0.5266
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7354 - loss: 0.5190 - val_accuracy: 0.7338 - val_loss: 0.5175
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7531 - loss: 0.5135 - val_accuracy: 0.7403 - val_loss: 0.5118
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7736 - loss: 0.4909 - val_accuracy: 0.7468 - 

### `Optimizer`

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):
  model = keras.Sequential()

  model.add(Input(shape=(8,)))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adagrad'])
  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
optimizer: adam
Score: 0.7857142686843872

Trial 1 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7467532753944397

Trial 2 summary
Hyperparameters:
optimizer: sgd
Score: 0.7077922224998474

Trial 3 summary
Hyperparameters:
optimizer: adagrad
Score: 0.6883116960525513


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8585 - loss: 0.3485 - val_accuracy: 0.7597 - val_loss: 0.5341
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8617 - loss: 0.3470 - val_accuracy: 0.7468 - val_loss: 0.5380
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8533 - loss: 0.3492 - val_accuracy: 0.7468 - val_loss: 0.5401
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8476 - loss: 0.3611 - val_accuracy: 0.7403 - val_loss: 0.5410
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8369 - loss: 0.3753 - val_accuracy: 0.7403 - val_loss: 0.5386
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8299 - loss: 0.4045 - val_accuracy: 0.7468 - val_loss: 0.5391
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8281 - loss: 0.3933 - val_accuracy: 0.7403 - val_loss: 0.5390
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8474 - loss: 0.3518 - val_accuracy: 0.7468

### `Neurons`

In [ ]:
def build_model(hp):

  model=Sequential()

  units = hp.Int('units',min_value=0,max_value=128,step=8)

  model.add(Input(shape=(8,)))
  model.add(Dense(units=units,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='my_dir')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 13s


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 96
Score: 0.7857142686843872

Trial 1 summary
Hyperparameters:
units: 128
Score: 0.7597402334213257

Trial 3 summary
Hyperparameters:
units: 64
Score: 0.7597402334213257

Trial 2 summary
Hyperparameters:
units: 16
Score: 0.7337662577629089

Trial 4 summary
Hyperparameters:
units: 8
Score: 0.6948052048683167


In [38]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [39]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 96)                  │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              97 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 961 (3.75 KB)

 Trainable params: 961 (3.75 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7648 - loss: 0.4846 - val_accuracy: 0.7987 - val_loss: 0.4964
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7537 - loss: 0.4824 - val_accuracy: 0.7922 - val_loss: 0.4948
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8008 - loss: 0.4403 - val_accuracy: 0.7792 - val_loss: 0.4975
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7924 - loss: 0.4486 - val_accuracy: 0.7662 - val_loss: 0.4969
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7842 - loss: 0.4581 - val_accuracy: 0.7662 - val_loss: 0.4969
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7853 - loss: 0.4497 - val_accuracy: 0.7597 - val_loss: 0.4975
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7725 - loss: 0.4621 - val_accuracy: 0.7857 - val_loss: 0.4994
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7588 - loss: 0.4455 - val_accuracy: 0.75

### `Layers`

In [46]:
def build_model(hp):
  model = keras.Sequential()

  model.add(Input(shape=(8,)))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(32,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


  return model

In [50]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='my_dir1')

In [51]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.649350643157959

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 25s


In [53]:
tuner.results_summary()

Results summary
Results in my_dir1/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 1
Score: 0.7727272510528564

Trial 0 summary
Hyperparameters:
num_layers: 6
Score: 0.649350643157959

Trial 2 summary
Hyperparameters:
num_layers: 2
Score: 0.649350643157959

Trial 3 summary
Hyperparameters:
num_layers: 10
Score: 0.649350643157959

Trial 4 summary
Hyperparameters:
num_layers: 3
Score: 0.649350643157959


In [54]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1}

In [55]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7351 - loss: 0.5369 - val_accuracy: 0.7662 - val_loss: 0.5240
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7461 - loss: 0.4969 - val_accuracy: 0.7922 - val_loss: 0.5147
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7597 - loss: 0.4987 - val_accuracy: 0.7792 - val_loss: 0.5094
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7750 - loss: 0.4809 - val_accuracy: 0.7792 - val_loss: 0.5083
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7697 - loss: 0.4756 - val_accuracy: 0.7792 - val_loss: 0.5070
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7658 - loss: 0.4932 - val_accuracy: 0.7792 - val_loss: 0.5075
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7618 - loss: 0.4746 - val_accuracy: 0.7857 - val_loss: 0.5069
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7743 - loss: 0.4595 - val_accuracy: 0.77

In [58]:
def build_model(hp):
    model = Sequential()
    counter = 0

    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        if counter == 0:
            model.add(Input(shape=(8,)))

        model.add(Dense(
            hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
            activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
        ))
        counter += 1

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adagrad']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [59]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='my_dir3',
                        project_name='final')

In [60]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.649350643157959

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 18s


In [61]:
tuner.results_summary()

Results summary
Results in my_dir3/final
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 7
units0: 104
activation0: relu
optimizer: rmsprop
units1: 16
activation1: relu
units2: 112
activation2: tanh
units3: 24
activation3: tanh
units4: 128
activation4: tanh
units5: 96
activation5: sigmoid
units6: 88
activation6: relu
units7: 8
activation7: sigmoid
units8: 40
activation8: tanh
Score: 0.7597402334213257

Trial 0 summary
Hyperparameters:
num_layers: 9
units0: 64
activation0: tanh
optimizer: adagrad
units1: 8
activation1: relu
units2: 8
activation2: relu
units3: 8
activation3: relu
units4: 8
activation4: relu
units5: 8
activation5: relu
units6: 8
activation6: relu
units7: 8
activation7: relu
units8: 8
activation8: relu
Score: 0.701298713684082

Trial 2 summary
Hyperparameters:
num_layers: 10
units0: 48
activation0: sigmoid
optimizer: sgd
units1: 64
activation1: relu
units2: 120
activation2: relu
units3: 24
activation3: si

In [62]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units0': 104,
 'activation0': 'relu',
 'optimizer': 'rmsprop',
 'units1': 16,
 'activation1': 'relu',
 'units2': 112,
 'activation2': 'tanh',
 'units3': 24,
 'activation3': 'tanh',
 'units4': 128,
 'activation4': 'tanh',
 'units5': 96,
 'activation5': 'sigmoid',
 'units6': 88,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'sigmoid',
 'units8': 40,
 'activation8': 'tanh'}

In [63]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 104)                 │             936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │           1,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 112)                 │           1,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 24)                  │           2,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │           3,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 96)                  │          12,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 88)                  │           8,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              89 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,441 (122.82 KB)

 Trainable params: 31,441 (122.82 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7833 - loss: 0.4636 - val_accuracy: 0.7662 - val_loss: 0.5437
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8031 - loss: 0.4169 - val_accuracy: 0.7208 - val_loss: 0.6064
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7656 - loss: 0.4517 - val_accuracy: 0.7338 - val_loss: 0.5364
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8302 - loss: 0.4176 - val_accuracy: 0.7597 - val_loss: 0.5683
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8320 - loss: 0.3785 - val_accuracy: 0.7208 - val_loss: 0.5527
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8263 - loss: 0.3968 - val_accuracy: 0.7273 - val_loss: 0.5865
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8358 - loss: 0.3941 - val_accuracy: 0.7143 - val_loss: 0.5905
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8273 - loss: 0.3847 - val_accuracy: 0.74